In [ ]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
import spacy_sentence_bert

In [ ]:
nlp = spacy_sentence_bert.load_model('en_stsb_roberta_large')

In [ ]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('explore')\
    .getOrCreate()
sc = spark.sparkContext._conf.setAll([('spark.driver.maxResultSize', '8g')])

In [ ]:
processed_df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')
processed_df.show()

In [ ]:
processed_df.count()

In [ ]:
def vectorize(text):
    return nlp(str(text)).vector

vectorize_udf = F.udf(lambda x: vectorize(x))
processed_df = processed_df.withColumn('vector', vectorize_udf(F.col('selected_text')))

In [ ]:
processed_df.write.mode('Overwrite').json('/common/users/shared/cs543_fall22_group3/combined/deep_vectors')